In [8]:
import requests
import pandas as pd

# Importar librerías para web scraping y manipulación de datos
# -----------------------------------------------------------------------
from bs4 import BeautifulSoup
import requests


from tqdm import tqdm

# Importar librerías para automatización de navegadores web con Selenium
# -----------------------------------------------------------------------
from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.

# Importar librerías para pausar la ejecución
# -----------------------------------------------------------------------
from time import sleep  # Sleep se utiliza para pausar la ejecución del programa por un número de segundos.

In [2]:
import re

In [9]:
pelis_imdb= pd.read_csv("caracteristicas_imdb.csv", index_col=0)

In [ ]:
lista_peliculas = [tuple(x) for x in pelis_imdb.values]

# Actores por peli

In [16]:
def diez_actores(lista_peliculas):  
    
    driver = webdriver.Chrome()

    driver.maximize_window()

    driver.get("https://www.imdb.com/")

    lista_tupla_actores = []
    
    sleep(3)
    
    for tupla in tqdm(lista_peliculas):
        
        driver.find_element("css selector", "#suggestion-search").send_keys(tupla[0], Keys.ENTER)
        sleep(3)
        
        lista_actores = []
        for i in range(1, 11):
            try:
                lista_actores.append(driver.find_element("css selector", f"#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-9178d6fe-1.fqVKQb.ipc-page-grid__item.ipc-page-grid__item--span-2 > section.ipc-page-section.ipc-page-section--base.sc-bfec09a1-0.jgUBLM.title-cast.title-cast--movie.celwidget > div.ipc-shoveler.ipc-shoveler--base.ipc-shoveler--page0.title-cast__grid > div.ipc-sub-grid.ipc-sub-grid--page-span-2.ipc-sub-grid--wraps-at-above-l.ipc-shoveler__grid > div:nth-child({i}) > div.sc-bfec09a1-7.gWwKlt > a").text)
            
            except:            
                break
        lista_actores.append(tupla[1])
        lista_tupla_actores.append(tuple(lista_actores))

        driver.back()

    driver.close()   
    
    return lista_tupla_actores


100%|██████████| 1/1 [00:08<00:00,  8.30s/it]


In [ ]:
actores_por_peli=diez_actores(lista_peliculas)

# Caracteristicas de los actores

In [ ]:
def caracteristicas_actores(actores_por_peli):

    driver = webdriver.Chrome()
    driver.maximize_window()

    driver.get("https://www.imdb.com/")
    lista_detalles_actores = []

    for tupla in actores_por_peli:
        
        for actor in tqdm(tupla[1:]):
        
            print(actor)
            sleep(7)
            driver.find_element("css selector", "#suggestion-search").send_keys(actor, Keys.ENTER)
                
            sleep(3)
            
            driver.find_element("css selector", "#__next > main > div.ipc-page-content-container.ipc-page-content-container--full.sc-872d7ac7-0.fqEQWL > div.ipc-page-content-container.ipc-page-content-container--center > section > div > div.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(3) > div.sc-17bafbdb-2.iUyFfD > ul > li:nth-child(1)").click()
            
            lista_caracteristicas = []
            #Sacamos año de nacimiento
            try:
                año = driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[3]/div[2]/div[2]/section/aside/div/span[2]').text.split(" ")[-1]
                    
                lista_caracteristicas.append(año)                  
                
            except:            
                lista_caracteristicas.append("Desconocido")
            
            #Sacamos el top de las peliculas por las que es conocido/a    

            try: 
                conocido = driver.find_element("xpath",'//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/div[3]/section[1]/div[2]').text
            except: 
                conocido="Sin datos"
            
            #Sacamos las profesiones     
            try:   
                que_hace = driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[2]/div/ul').text.replace("\n", ', ')
            
            except:
                que_hace="Sin datos"
            
            try: 
                oscar= driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[1]/div/ul/li/a[1]').text
                if "Ganó" in oscar: 
                    lista_caracteristicas.append(oscar)
                else: 
                    lista_caracteristicas.append("Nominado")
            except: 
                lista_caracteristicas.append("No tiene galardones ni nominaciones")
                

            lista_caracteristicas.append(actor)  
            lista_caracteristicas.append(conocido.split("\n")[::4])
            lista_caracteristicas.append(que_hace)
            lista_detalles_actores.append(tuple(lista_caracteristicas))
            driver.back()
        
    driver.close()
    
    return lista_detalles_actores  


In [ ]:
caracteristicas_imdb_actores=caracteristicas_actores(actores_por_peli)

## Limpiar la tupla del resultado que nos devuelve el codigo de caracteristicas

In [ ]:
#Una vez que tengamos la lista de las caracteristicas de los actores, tendremos que 
#Transformar la lista de tuplas en lista de listas 
lista_limpieza = [list(x) for x in caracteristicas_imdb_actores]

In [ ]:
#Eliminamos la lista de 'por que es conocido' y lo transformamos a strings. Recordad que la lista limpieza
#es la transformacion del paso anterior.
for indice,actriz in enumerate(lista_limpieza): 
    for elemento in actriz:
        if type(elemento)==list:
            elemento_limpio= ", ".join(elemento)
            lista_limpieza[indice][3] = elemento_limpio
            print(elemento_limpio)

In [ ]:
#Convertimos nuevamente la lista de listas en lista de tuplas
lista_limpieza_tupla = [tuple(x) for x in lista_limpieza]

In [ ]:
#Este resultado es el que guardamos en csv para la base de datos
lista_limpieza_tupla

In [ ]:
df_caracterisiticas=pd.DataFrame(lista_limpieza_tupla)

In [ ]:
df_caracterisiticas.to_csv("caracteristicas_actores.csv")